## Import NCES CCD (Common Core Data) Enrollments from text files

When trying to import the non-text-file versions of CCD enrollments, many years failed, seemingly due to a poor Python implementation of the file format import.

This file imports all years from the text file versions.

In [1]:
### Boilerplate and imports

import BeautifulSoup, csv, glob, json, os, math, numbers, pandas, re, scipy, scipy.sparse, shutil
import subprocess, sys, thread, threading, time, urllib2

import pandas as pd
import numpy as np

#!pip install BeautifulSoup

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [2]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))
pd.set_option('display.max_columns', 500)

In [3]:
# Per-school Common Core Data (CCD) from NCES
# https://nces.ed.gov/ccd/pubschuniv.asp

In [4]:
ccd_download_dir = 'allegheny_county/schools/nces'

def mirror_ccd_file(src_url):
    dest = ccd_download_dir + '/' + os.path.basename(src_url).replace(' ', '_')
    download_file(src_url.replace(' ', '%20'), dest)
    return dest

def mirror_ccd_unzip(src_url):
    zipfile = mirror_ccd_file(src_url)
    unzip_file(zipfile)
    targets = glob.glob(os.path.splitext(zipfile)[0] + '/*')
    assert len(targets) == 1
    return targets[0]

In [5]:
url_prefix = 'https://nces.ed.gov/ccd/'

download_page = urllib2.urlopen(url_prefix + 'pubschuniv.asp').read()

links = [link.get('href') for link in BeautifulSoup.BeautifulSoup(download_page).findAll('a') if link.get('href')]

In [6]:
def datafile_urls(year):
    if year <= 2013:
        pattern = r'data/zip/[a-z]+%02d\d?[a-z]*_(dat|txt|supp_txt)\.zip' % (year % 100)
    else:
        pattern = r'data/zip/.*?(_(%02d|%04d)%02d_).*_(txt|csv)\.zip' % (year % 100, year, (year + 1) % 100)
    ret = [url_prefix + link for link in links if re.match(pattern, link, re.IGNORECASE)]
    if year == 2015:
        # 2015 geocodes aren't on the CCD download page
        ret.append('https://nces.ed.gov/programs/edge/data/EDGE_GEOCODE_PUBLICSCH_1516.zip')
    return ret

def layout_urls(year):
    if year <= 2013:
        pattern = r'data/txt/[a-z]+%02d\d?[a-z]?lay.txt' % (year % 100)
    else:
        pattern = r'xls/%04d-%02d.*?layout.xlsx' % (year, (year + 1) % 100)
    return [url_prefix + link for link in links if re.match(pattern, link, re.IGNORECASE)]

def data_files(year):
    return [mirror_ccd_unzip(url) for url in datafile_urls(year)]

def layout_files(year):
    return [mirror_ccd_file(url) for url in layout_urls(year)]

#for year in range(1986, 2018):
#    for link in layout_links(year):
#        print layout_files(year)

# https://nces.ed.gov/programs/edge/data/EDGE_GEOCODE_PUBLICSCH_1516.zip
datafile_urls(2015)

[u'https://nces.ed.gov/ccd/Data/zip/ccd_sch_029_1516_w_2a_011717_csv.zip',
 u'https://nces.ed.gov/ccd/Data/zip/ccd_sch_052_1516_w_2a_011717_csv.zip',
 u'https://nces.ed.gov/ccd/Data/zip/ccd_sch_059_1516_w_2a_011717_csv.zip',
 u'https://nces.ed.gov/ccd/Data/zip/ccd_sch_129_1516_w_2a_011717_csv.zip',
 u'https://nces.ed.gov/ccd/Data/zip/ccd_sch_033_1516_w_2a_011717_csv.zip',
 'https://nces.ed.gov/programs/edge/data/EDGE_GEOCODE_PUBLICSCH_1516.zip']

In [9]:
def merge_cols(a, b):
    ret = a.copy()
    use_b = a.isna() & ~b.isna()
    ret[use_b] = b[use_b]
    return ret

def merge_ccd_dataframes(frames, join_col = 'NCESSCH'):
    m = frames[0]
    for rhs in frames[1:]:
        cols_to_merge = sorted(set(m.columns) & set(rhs.columns) - set([join_col]))
        m = pd.merge(m, rhs, on=join_col, how='outer', validate='1:1', suffixes=('_xxx', '_yyy'))
        for col in cols_to_merge:
            m[col] = merge_cols(m[col + '_xxx'], m[col + '_yyy'])
            m.drop(columns = [col + '_xxx', col + '_yyy'], inplace=True)
    return m

def read_ccd(year):
    srcs = ccd_src_from_year(year)

def read_and_merge_ccd_srcs(srcs):
    return merge_ccd_dataframes([read_ccd_table(src) for src in srcs], 'NCESSCH')

In [10]:
def compute_year_pre_2007(year):
    layout = layout_files(year)[0]

    colspecs = []
    names = []
    dtypes = {}

    for line in open(layout):
        
        if year <= 1995 and re.match(r'NCESSHC\s', line):
            line = line.replace('NCESSHC', 'NCESSCH')

        if not re.match(r'NCESSCH\s', line) and not names:
            continue
        if re.match(r'\w+', line) or re.match(r'\s*\+\w+', line):
            if year >= 1998:
                (name, begin, end, length, dtype) = line.split()[0:5]
            else:
                (name, dtype, begin_end) = line.split()[0:3]
                (begin, end) = begin_end.split('-')
            #print [name, begin, end, length, dtype]
            begin = int(begin) - 1
            if end[-1] == '*':
                end = end[:-1]
            end = int(end)
            if name[0] == '+':
                name = name[1:]
                
            # patch broken file
            if year == 2004 and name == 'IAM12F04':
                dtype = 'AN'
                
            if year <= 2001:
                # many values from 2000 and 2001 don't parse numerically
                dtype = 'AN'
            
            if dtype == 'AN':
                dtype = np.str
            elif dtype == 'N':
                dtype = np.float64
            else:
                raise Exception("Unknown field type %s\nFrom line %s\nIn layout file %s" % (dtype, line, layout))
            #print name, begin, end, dtype
            names.append(name)
            colspecs.append((begin, end))
            dtypes[name] = dtype

    #print names
    assert 'NCESSCH' in names
    assert 'LEAID' in names
    assert dtypes['NCESSCH'] == np.str
    assert dtypes['LEAID'] == np.str
    assert colspecs[0][0] == 0
    
    dataframes = []
    for data_file in data_files(year):
        dataframes.append(pd.read_fwf(data_file, header=None, names=names, colspecs=colspecs, dtype=dtypes))
    
    return pd.concat(dataframes)

In [20]:
def uniqpath():
    return '%s-%s' % (thread.get_ident(), os.getpid())

def read_src(filename, year):
    dtypes = {'NCESSCH': np.str, 'LEAID': np.str}
    if os.path.isdir(filename):
        # Look for something good inside
        filename = glob.glob(filename + '/*.xlsx')[0]
    suffix = os.path.splitext(filename)[1]
    if suffix == '.xlsx':
        return pd.read_excel(filename, dtype=dtypes)
    else:
        # Decide on comma or tab as delimiter
        if open(filename).read().count("\t") > 100:
            sep = '\t'
        else:
            sep = ','
        return pd.read_csv(filename, sep=sep, dtype=dtypes)

def compute_year(year):
    if year < 2007:
        return compute_year_pre_2007(year)
    else:
        dataframes = [read_src(data_file, year) for data_file in data_files(year)]
        merged = merge_ccd_dataframes(dataframes)
        return merged

def compute_year_if_needed(year, force_recompute=False):
    filename = ccd_download_dir + ('/%d-ccd-computed-v2.h5' % year)
    if os.path.exists(filename) and not force_recompute:
        print '%s already exists, skipping' % filename
        return
    ccd = compute_year(year)
    tmp = filename + uniqpath() + '.h5'
    ccd.to_hdf(tmp, 'data')
    os.rename(tmp, filename)
    print 'Created %s' % filename

force_recompute = True

for year in range(2015, 2018):
    compute_year_if_needed(year, force_recompute=force_recompute)




allegheny_county/schools/nces/ccd_sch_029_1516_w_2a_011717_csv.zip already downloaded
allegheny_county/schools/nces/ccd_sch_029_1516_w_2a_011717_csv.zip already unzipped
allegheny_county/schools/nces/ccd_sch_052_1516_w_2a_011717_csv.zip already downloaded
allegheny_county/schools/nces/ccd_sch_052_1516_w_2a_011717_csv.zip already unzipped
allegheny_county/schools/nces/ccd_sch_059_1516_w_2a_011717_csv.zip already downloaded
allegheny_county/schools/nces/ccd_sch_059_1516_w_2a_011717_csv.zip already unzipped
allegheny_county/schools/nces/ccd_sch_129_1516_w_2a_011717_csv.zip already downloaded
allegheny_county/schools/nces/ccd_sch_129_1516_w_2a_011717_csv.zip already unzipped
allegheny_county/schools/nces/ccd_sch_033_1516_w_2a_011717_csv.zip already downloaded
allegheny_county/schools/nces/ccd_sch_033_1516_w_2a_011717_csv.zip already unzipped
allegheny_county/schools/nces/EDGE_GEOCODE_PUBLICSCH_1516.zip already downloaded
allegheny_county/schools/nces/EDGE_GEOCODE_PUBLICSCH_1516.zip already

/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:43: DtypeWarning: Columns (6,14,22) have mixed types. Specify dtype option on import or set low_memory=False.
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:43: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.


reading xlsx


/home/rsargent/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:279: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  f(store)
/home/rsargent/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['NCESSCH', 'MSTREET1', 'MSTREET2', 'MSTREET3', 'MCITY', 'MSTATE', 'LSTREET1', 'LSTREET2', 'LSTREET3', 'OUT_OF_STATE_FLAG', 'SCH_TYPE_TEXT', 'RECON_STATUS', 'GSLO', 'GSHI', 'LEVEL', 'VIRTUAL', 'BIES', 'SY_STATUS_TEXT', 'UPDATED_STATUS_TEXT', 'EFFECTIVE_DATE', 'CHARTER_TEXT', 'PKOFFERED', 'KGOFFERED', 'G1OFFERED', 'G2OFFERED', 'G3OFFERED', 'G4OFFERED', 'G5OFFERED', 'G6OFFERED', 'G7OFFERED', 'G8OFFERED', 'G9OFFERED', 'G10OFFERED', 'G11OFFERED', 'G12OFFERED', 'G13OFFER

Created allegheny_county/schools/nces/2015-ccd-computed-v2.h5
allegheny_county/schools/nces/ccd_sch_029_1617_w_0e_050317_csv.zip already downloaded
allegheny_county/schools/nces/ccd_sch_029_1617_w_0e_050317_csv.zip already unzipped


/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:43: DtypeWarning: Columns (14,15,21,22,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.


Created allegheny_county/schools/nces/2016-ccd-computed-v2.h5
allegheny_county/schools/nces/ccd_sch_029_1718_w_0a_03302018_csv.zip already downloaded
allegheny_county/schools/nces/ccd_sch_029_1718_w_0a_03302018_csv.zip already unzipped
Created allegheny_county/schools/nces/2017-ccd-computed-v2.h5


In [23]:
pd.read_hdf('allegheny_county/schools/nces/2017-ccd-computed-v2.h5')

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE,SCH_TYPE_TEXT,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2017-2018,1,ALABAMA,AL,Albertville Middle School,Albertville City,1,NaN,AL-101,0100005,AL-101-0010,010000500870,100870,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,2336.0,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,NaN,(256)878-2341,http://www.albertk12.org,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,07,08,Middle,As reported
1,2017-2018,1,ALABAMA,AL,Albertville High School,Albertville City,1,NaN,AL-101,0100005,AL-101-0020,010000500871,100871,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2399.0,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2322.0,(256)894-5000,http://www.albertk12.org,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,09,12,High,As reported
2,2017-2018,1,ALABAMA,AL,Evans Elementary School,Albertville City,1,NaN,AL-101,0100005,AL-101-0110,010000500879,100879,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1300.0,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1346.0,(256)878-7698,http://www.albertk12.org,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,05,06,Middle,As reported
3,2017-2018,1,ALABAMA,AL,Albertville Elementary School,Albertville City,1,NaN,AL-101,0100005,AL-101-0200,010000500889,100889,1100 Horton Rd,NaN,NaN,Albertville,AL,35950,2532.0,1100 Horton Rd,NaN,NaN,Albertville,AL,35950,NaN,(256)894-4822,http://www.albertk12.org,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,03,04,Elementary,As reported
4,2017-2018,1,ALABAMA,AL,Big Spring Lake Kindergarten School,Albertville City,1,NaN,AL-101,0100005,AL-101-0035,010000501616,101616,257 Country Club Rd,NaN,NaN,Albertville,AL,35951,3927.0,257 Country Club Rd,NaN,NaN,Albertville,AL,35951,NaN,(256)878-7922,http://www.albertk12.org,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,KG,KG,Elementary,As reported
5,2017-2018,1,ALABAMA,AL,Albertville Primary School,Albertville City,1,NaN,AL-101,0100005,AL-101-0005,010000502150,102150,1100 Horton Rd,NaN,NaN,Albertville,AL,35950,2532.0,1100 Horton Road,NaN,NaN,Albertville,AL,35950,NaN,(256)878-6611,http://www.albertk12.org,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,Yes,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,01,02,Elementary,As reported
6,2017-2018,1,ALABAMA,AL,Kate Duncan Smith DAR Middle,Marshall County,1,NaN,AL-048,0100006,AL-048-0143,010000600193,100193,6077 Main St,NaN,NaN,Grant,AL,35747,8333.0,6077 Main St,NaN,NaN,Grant,AL,35747,NaN,(256)728-5950,http://darmiddle.marshallk12.org/,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No,05,08,Middle,As reported
7,2017-2018,1,ALABAMA,AL,Asbury High School,Marshall County,1,NaN,AL-048,0100006,AL-048-0030,010000600872,100872,1990 Asbury Rd,NaN,NaN,Albertville,AL,35951,6040.0,1990 Asbury Rd,NaN,NaN,Albertville,AL,35951,NaN,(256)878-4068,http://www.marshallk12.org,1,Open,1,Open,03/30/2018,1,Regular School,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,06,12,Other,As reported
8,2017-2018,1,ALABAMA,AL,Clay

In [241]:
pd.read_excel('allegheny_county/schools/nces/EDGE_GEOCODE_PUBLICSCH_1516/EDGE_GEOCODE_PUBLICSCH_1516/EDGE_GEOCODE_PUBLICSCH_1516.xlsx', dtype={'NCESSCH':np.str})

,NCESSCH,NAME,OPSTFIPS,LSTREE,LCITY,LSTATE,LZIP,LZIP4,STFIP15,CNTY15,NMCNTY15,LOCALE15,LAT1516,LON1516,CBSA15,NMCBSA15,CBSATYPE15,CSA15,NMCSA15,NECTA15,NMNECTA15,CD15,SLDL15,SLDU15
0,010000200277,Sequoyah Sch - Chalkville Campus,1,1000 Industrial School Road,Birmingham,AL,35220,M,01,01073,Jefferson County,21,33.673661,-86.628755,13820,"Birmingham-Hoover, AL",1,142,"Birmingham-Hoover-Talladega, AL",N,N,0106,044,020
1,010000201667,Camps,1,1601 County Rd. 57,Prattville,AL,36067,M,01,01001,Autauga County,41,32.521681,-86.530132,33860,"Montgomery, AL",1,N,N,N,N,0102,042,030
2,010000201670,Det Ctr,1,2109 Bashi Rd Bldg 509,Thomasville,AL,36784,M,01,01025,Clarke County,41,31.938444,-87.750529,N,N,N,N,N,N,N,0107,068,024
3,010000201705,Wallace Sch - Mt Meigs Campus,1,1000 Industrial School Road,Mount Meigs,AL,36057,M,01,01101,Montgomery County,41,32.374812,-86.082360,33860,"Montgomery, AL",1,N,N,N,N,0103,075,025
4,010000201706,McNeel Sch - Vacca Campus,1,8950 Roebuck Blvd,Birmingham,AL,35206,M,01,01073,Jefferson County,12,33.583385,-86.710058,13820,"Birmingham-Hoover, AL",1,142,"Birmingham-Hoover-Talladega, AL",N,N,0107,058,020
5,010000201876,Alabama Youth Services,1,Central Office Bldg,Mt Meigs,AL,36057,M,01,01101,Montgomery County,41,32.374847,-86.082332,33860,"Montgomery, AL",1,N,N,N,N,0103,075,025
6,010000500870,Ala Avenue Middle Sch,1,600 E Alabama Ave,Albertville,AL,35950,M,01,01095,Marshall County,32,34.260194,-86.206174,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009
7,010000500871,Albertville High Sch,1,402 E McCord Ave,Albertville,AL,35950,M,01,01095,Marshall County,32,34.262154,-86.204863,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009
8,010000500879,Evans Elem Sch,1,901 W McKinney Ave,Albertville,AL,35950,M,01,01095,Marshall County,32,34.273161,-86.220086,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009
9,010000500889,Albertville Elem Sch,1,1100 Horton Rd,Albertville,AL,35950,M,01,01095,Marshall County,32,34.253251,-86.221834,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL",N,N,0104,026,009


In [100]:
z=pd.read_hdf('allegheny_county/schools/nces/2013-ccd-computed-v2.h5')
z

,SURVYEAR,NCESSCH,FIPST,LEAID,SCHNO,STID,SEASCH,LEANM,SCHNAM,PHONE,MSTREE,MCITY,MSTATE,MZIP,MZIP4,LSTREE,LCITY,LSTATE,LZIP,LZIP4,TYPE,STATUS,UNION,ULOCAL,LATCOD,LONCOD,CONUM,CONAME,CDCODE,BIES,RECONSTY,RECONSTF,FTE,GSLO,GSHI,PKOFFRD,KGOFFRD,G01OFFRD,G02OFFRD,G03OFFRD,G04OFFRD,G05OFFRD,G06OFFRD,G07OFFRD,G08OFFRD,G09OFFRD,G10OFFRD,G11OFFRD,G12OFFRD,UGOFFRD,LEVEL,TITLEISTAT,TITLEI,STITLI,MAGNET,CHARTR,SHARED,FRELCH,REDLCH,TOTFRL,PK,AMPKM,AMPKF,ASPKM,ASPKF,HIPKM,HIPKF,BLPKM,BLPKF,WHPKM,WHPKF,HPPKM,HPPKF,TRPKM,TRPKF,KG,AMKGM,AMKGF,ASKGM,ASKGF,HIKGM,HIKGF,BLKGM,BLKGF,WHKGM,WHKGF,HPKGM,HPKGF,TRKGM,TRKGF,G01,AM01M,AM01F,AS01M,AS01F,HI01M,HI01F,BL01M,BL01F,WH01M,WH01F,HP01M,HP01F,TR01M,TR01F,G02,AM02M,AM02F,AS02M,AS02F,HI02M,HI02F,BL02M,BL02F,WH02M,WH02F,HP02M,HP02F,TR02M,TR02F,G03,AM03M,AM03F,AS03M,AS03F,HI03M,HI03F,BL03M,BL03F,WH03M,WH03F,HP03M,HP03F,TR03M,TR03F,G04,AM04M,AM04F,AS04M,AS04F,HI04M,HI04F,BL04M,BL04F,WH04M,WH04F,HP04M,HP04F,TR04M,TR04F,G05,AM05M,AM05F,AS05M,AS05F,HI05M,HI05F,BL05M,BL05F,WH05M,WH05F,HP05M,HP05F,TR05M,TR05F,G06,AM06M,AM06F,AS06M,AS06F,HI06M,HI06F,BL06M,BL06F,WH06M,WH06F,HP06M,HP06F,TR06M,TR06F,G07,AM07M,AM07F,AS07M,AS07F,HI07M,HI07F,BL07M,BL07F,WH07M,WH07F,HP07M,HP07F,TR07M,TR07F,G08,AM08M,AM08F,AS08M,AS08F,HI08M,HI08F,BL08M,BL08F,WH08M,WH08F,HP08M,HP08F,TR08M,TR08F,G09,AM09M,AM09F,AS09M,AS09F,HI09M,HI09F,BL09M,BL09F,WH09M,WH09F,HP09M,HP09F,TR09M,TR09F,G10,AM10M,AM10F,AS10M,AS10F,HI10M,HI10F,BL10M,BL10F,WH10M,WH10F,HP10M,HP10F,TR10M,TR10F,G11,AM11M,AM11F,AS11M,AS11F,HI11M,HI11F,BL11M,BL11F,WH11M,WH11F,HP11M,HP11F,TR11M,TR11F,G12,AM12M,AM12F,AS12M,AS12F,HI12M,HI12F,BL12M,BL12F,WH12M,WH12F,HP12M,HP12F,TR12M,TR12F,UG,AMUGM,AMUGF,ASUGM,ASUGF,HIUGM,HIUGF,BLUGM,BLUGF,WHUGM,WHUGF,HPUGM,HPUGF,TRUGM,TRUGF,MEMBER,AM,AMALM,AMALF,ASIAN,ASALM,ASALF,HISP,HIALM,HIALF,BLACK,BLALM,BLALF,WHITE,WHALM,WHALF,PACIFIC,HPALM,HPALF,TR,TRALM,TRALF,TOTETH,VIRTUALSTAT,NSLPSTATUS,CHARTAUTH1,CHARTAUTH2
0,2013,010000200277,1,100002,277,210,20,ALABAMA YOUTH SERVICES,SEQUOYAH SCH - CHALKVILLE CAMPUS,2056808574,P O BOX 9486,BIRMINGHAM,AL,35220,486.0,RT 2 OLD SPRINGVILLE RD,PINSON,AL,36126,486.0,4,1,0,21,33.673727,-86.628716,1073,JEFFERSON COUNTY,106,2,N,2,-1.00,07,12,2,2,2,2,2,2,2,2,1,1,1,1,1,1,2,3,M,M,M,2,N,2,-1,-1,-1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,VIRTUALNO,M,N,N
1,2013,010000201402,1,100002,1402,210,25,ALABAMA YOUTH SERVICES,EUFAULA SCH - EUFAULA CAMPUS,3346874441,315 OUTBACK ROAD,CLAYTON,AL,36016,NaN,315 OUTBACK RD,CLAYTON,AL,36016,NaN,4,1,0,42,31.881600,-85.252800,1005,BARBOUR COUNTY,102,2,N,2,-2.00,03,12,2,2,2,2,1,1,1,1,1,1,1,1,1,1,2,4,M,M,M,2,N,2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-

In [15]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu86lay.txt')
mirror_ccd_unzip('https://nces.ed.gov/ccd/data/zip/psu86ai_dat.zip', 'txt')

!cat allegheny_county/schools/nces/psu86lay.txt

allegheny_county/schools/nces/psu86lay.txt already downloaded
allegheny_county/schools/nces/psu86ai_dat.zip already downloaded
allegheny_county/schools/nces/psu86ai_dat.zip already unzipped



















































In [16]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu87lay.txt')

!cat allegheny_county/schools/nces/psu87lay.txt

Done, wrote 2436 bytes to allegheny_county/schools/nces/psu87lay.txt
        Common Core of Data Public School Universe 1987 - 1988
                	      LRECL=274
     		  
	  (*) Fields have one implied decimal place.
	  
	  (+) Fields represent sub-fields of the fields immediately 
	      preceding them.  
	  
NOTE: The variable 'YEAR' has been added to this data set and reflects the
school year (fall) this data was collected.
                                    
Name     	Type 	Position	Size	Description    	

NCESSCH		AN	001-012		12	NCES Unique Identifier
  +FIPS  	 N	001-002		2	FIPS State Code for Location of School
  +LEAID	 N	001-007		7	NCES Agency ID
  +SCHNO	 N	008-012		5	NCES School ID	
STID87  	AN	013-026		14	State Agency ID
LEANM87		AN	027-056		30	Agency Name
SEASCH87	AN	057-076      	20	State School ID	
SCHNAM87	AN	077-106		30	School Name    	
STREET87	AN	107-131		25	Address        	
CITY87 		AN	132-149		18	City           	
ST87		AN	150-151		2	State         	
ZIP87	  	AN	

In [17]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu90lay.txt')

!cat allegheny_county/schools/nces/psu90lay.txt

Done, wrote 2481 bytes to allegheny_county/schools/nces/psu90lay.txt
	Common Core of Data Public School Universe 1990 - 1991		
 			 LRECL=298
       	        
	      (*) Fields have one implied decimal place.
	  
	      (+) Fields represent sub-fields of the fields immediately 
	      preceding them.  

NOTE: The varialbe 'YEAR' has been added to this data set and reflects the
school year (fall) this data was collected.
 
Name     	Type 	Position	Size		Description
   
NCESSCH		AN	001-012 	12		NCES Unique Identifier
  +FIPS 	N	001-002		2		FIPS State Code for Location of School
  +LEAID	N	001-007		7		NCES Agency ID
  +SCHNO	N	008-012		5		NCES School ID
STID90  	AN	013-026		14		State Agency ID
LEANM90		AN	027-056		30		Agency Name
SEASCH90	AN	057-076		20		State School ID	
SCHNAM90  	AN	077-106		30		School Name    	
STREET90	AN	107-131		25		Address        	
CITY90		AN	132-149		18		City           
ST90		AN	150-151		2		State         
ZIP90	  	AN	152-156		5		Zip          
ZIP490		AN	157-160		4

In [18]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu95lay.txt')

!cat allegheny_county/schools/nces/psu95lay.txt

Done, wrote 5786 bytes to allegheny_county/schools/nces/psu95lay.txt
         Common Core of Data Public School Universe 1995-96
                           (LRECL = 336)
              
	  (*) Fields have one implied decimal place.
	  
	  (+) Fields represent sub-fields of the fields immediately 
	      preceding them. 
 
**Please Note: "FIPS" stands for Federal Information Processing Standard (see State Codes
  and Abbreviations in the General Documentation)

Name    	Type   Position       Size           Description

NCESSHC   	AN   	001-012         12            UNIQUE SCHOOL ID (NCES ASSIGNED)
  +FIPS     	AN   	001-002          2            FIPS STATE CODE FOR LOCATION OF SCHOOL
  +LEAID     	AN   	001-007          7            UNIQUE SYSTEM ID (NCES ASSIGNED)
  +SCHNO     	AN   	008-012          5            SCHOOL NUMBER WITHIN STATE (NCES)
STID95    	AN   	013-026         14            STATE AGENCY ID
LEANM95   	AN   	027-056         30            NAME OF OPERATING AGENCY
SEASCH9

In [19]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu00lay.txt')

!cat allegheny_county/schools/nces/psu00lay.txt

Done, wrote 102628 bytes to allegheny_county/schools/nces/psu00lay.txt
Common Core of Data, Public Elementary/Secondary School Universe Survey, 2000-2001

FILE NAME = sc001a.dat, 2000-2001 (3 files provided:  A-I states, K-N states, and O-W states)

NVAR = 574

NOBS = 96570

LRECL = 1692

(*) Fields have one explicit decimal place

(+) Fields represent sub-fields of the fields immediately preceding them.

The file contains data for the school year 2000-2001 sorted by the NCES assigned school identification code (NCESSCH).

Variable    Start   End     Field   Data
Name        Pos.    Pos.    Length  Type    Description

NCESSCH     0001    0012    12      AN      ID assigned by NCES to each school.
   +FIPST   0001    0002    2       AN      Federal Information Processing Standards, FIPS state code.
   +LEAID   0001    0007    7       AN      ID assigned by NCES to system.  NOTE: Position # 0001-0002 is the FIPS state code for the
                                            location of 

In [22]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu051alay.txt')

!cat allegheny_county/schools/nces/psu051alay.txt

allegheny_county/schools/nces/psu051alay.txt already downloaded










































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [25]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu102alay.txt')

!cat allegheny_county/schools/nces/psu102alay.txt

Done, wrote 41557 bytes to allegheny_county/schools/nces/psu102alay.txt
Common Core of Data, Public Elementary/Secondary School Universe Survey, 2010-11

FILE NAME = sc102a.txt, 2010-11

NVAR = 308

NOBS = 103,813

(*) Fields have one explicit decimal place, unless otherwise noted.

(+) Fields represent subfields of the fields immediately preceding them.

The file contains data for the school year 2010�11 sorted by the NCES assigned school identification code (NCESSCH). Data
presented in this release are available in two formats�SAS datasets (.sas7bdat) and tab-separated values text files (.txt).

Note that starting with the 2002�03 CCD, all numeric fields contain a -1 to indicate missing and a -2 to indicate not applicable.
Starting with the 2009-10 2a file, the numeric fields contain a -9 to indicate that the data do not meet NCES data quality standards.
Previously, numeric fields contained an �M� to indicate missing and an �N� to indicate not applicable.  Character fields continue
t

In [27]:
mirror_ccd_unzip('https://nces.ed.gov/ccd/Data/zip/sc102a_txt.zip', 'txt')
!head allegheny_county/schools/nces/sc102a_txt/sc102a.txt

allegheny_county/schools/nces/sc102a_txt.zip already downloaded
allegheny_county/schools/nces/sc102a_txt.zip already unzipped












In [ ]:
mirror_ccd_file('https://nces.ed.gov/ccd/data/txt/psu152alay.txt')

!cat allegheny_county/schools/nces/psu152alay.txt

In [ ]:
    hdffile = os.path.splitext(sasfile)[0] + '.h5'
    if os.path.exists(hdffile):
        print 'Reading %s' % hdffile
        return pd.read_hdf(hdffile)
    else:
        print 'Reading %s' % sasfile
        ret = pd.read_sas(sasfile)
        print 'Writing %s' % hdffile
        tmp = hdffile + uniqpath() + '.tmp.h5'
        ret.to_hdf(tmp, 'data')
        os.rename(tmp, hdffile)
        return ret
    mirror_ccd_src('https://nces.ed.gov/ccd/data/zip/psu86ai_dat.zip')

In [3]:


# Reads src_url into table, via intermediate caches of downloading original source and converting to local HDF5
def read_ccd_src(src_url):
    dest = ccd_download_dir + '/' + os.path.basename(src_url)
    download_file(src_url, dest)
    unzip_file(dest)
    sasfile = glob.glob(os.path.splitext(dest)[0] + '/*.sas7bdat')[0]
    hdffile = os.path.splitext(sasfile)[0] + '.h5'
    if os.path.exists(hdffile):
        print 'Reading %s' % hdffile
        return pd.read_hdf(hdffile)
    else:
        print 'Reading %s' % sasfile
        ret = pd.read_sas(sasfile)
        print 'Writing %s' % hdffile
        tmp = hdffile + uniqpath() + '.tmp.h5'
        ret.to_hdf(tmp, 'data')
        os.rename(tmp, hdffile)
        return ret
    
def ccd_from_year(year):
    src = ccd_src_from_year(year)

In [ ]:
def compute_2007(): return read_ccd_srcs()

In [106]:
def compute_2017(): return read_ccd_src('https://nces.ed.gov/ccd/Data/zip/ccd_sch_029_1718_w_0a_03302018_sas.zip')

def compute_2016(): return read_ccd_src('https://nces.ed.gov/ccd/Data/zip/ccd_sch_029_1617_w_0e_050317_sas.zip')

def compute_2015(): return read_and_merge_ccd_srcs([
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_029_1516_w_2a_011717_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_052_1516_w_2a_011717_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_059_1516_w_2a_011717_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_129_1516_w_2a_011717_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_033_1516_w_2a_011717_sas.zip'])

def compute_2014(): return read_and_merge_ccd_srcs([
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_029_1415_w_0216161a_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_052_1415_w_0216161a_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_059_1415_w_0216161a_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_129_1415_w_0216161a_sas.zip',
    'https://nces.ed.gov/ccd/Data/zip/ccd_sch_033_1415_w_0216161a_sas.zip'])

def compute_2013(): return read_ccd_src('https://nces.ed.gov/ccd/Data/zip/sc132a_sas.zip')

def compute_2012(): return read_ccd_src('https://nces.ed.gov/ccd/Data/zip/sc122a_sas.zip')

def compute_2011(): return read_ccd_src('https://nces.ed.gov/ccd/Data/zip/sc111a_supp_sas.zip')

def compute_2010(): return read_ccd_src('https://nces.ed.gov/ccd/Data/zip/sc102a_sas7bdat.zip')

def compute_2009(): return read_ccd_src('https://nces.ed.gov/ccd/data/zip/sc092a_sas.zip')

def compute_2008(): return read_ccd_src('https://nces.ed.gov/ccd/data/zip/sc081b_sas.zip')

def compute_2007(): return read_ccd_src('https://nces.ed.gov/ccd/data/zip/sc071b_sas.zip')

#def compute_2006(): return read_ccd_src('')

#def compute_2005(): return read_ccd_src('')

#def compute_2004(): return read_ccd_src('')

#def compute_2003(): return read_ccd_src('')




    


In [115]:
def compute_if_needed(year):
    filename = ccd_download_dir + ('/%d-ccd-computed.h5' % year)
    if os.path.exists(filename):
        print '%s already exists, skipping' % filename
        return
    ccd = eval('compute_%d()' % year)
    tmp = filename + uniqpath() + '.h5'
    ccd.to_hdf(tmp, 'data')
    os.rename(tmp, filename)
    print 'Created %s' % filename

for year in range(2017, 1980, -1):
    try:
        compute_if_needed(year)
    except:
        print '%d failed' % year
    

allegheny_county/schools/nces/2017-ccd-computed.h5 already exists, skipping
allegheny_county/schools/nces/2016-ccd-computed.h5 already exists, skipping
allegheny_county/schools/nces/2015-ccd-computed.h5 already exists, skipping
allegheny_county/schools/nces/ccd_sch_029_1415_w_0216161a_sas.zip already downloaded
allegheny_county/schools/nces/ccd_sch_029_1415_w_0216161a_sas.zip already unzipped
Reading allegheny_county/schools/nces/ccd_sch_029_1415_w_0216161a_sas/ccd_sch_029_1415_w_0216161a.h5
allegheny_county/schools/nces/ccd_sch_052_1415_w_0216161a_sas.zip already downloaded
allegheny_county/schools/nces/ccd_sch_052_1415_w_0216161a_sas.zip already unzipped
Reading allegheny_county/schools/nces/ccd_sch_052_1415_w_0216161a_sas/ccd_sch_052_1415_w_0216161a.sas7bdat
2014 failed
allegheny_county/schools/nces/2013-ccd-computed.h5 already exists, skipping
allegheny_county/schools/nces/2012-ccd-computed.h5 already exists, skipping
allegheny_county/schools/nces/sc111a_supp_sas.zip already downlo

In [112]:
!file allegheny_county/schools/nces/ccd_sch_052_1415_w_0216161a_sas/*

allegheny_county/schools/nces/ccd_sch_052_1415_w_0216161a_sas/ccd_sch_052_1415_w_0216161a.sas7bdat: SAS 7+ data file


In [110]:
pd.read_sas('allegheny_county/schools/nces/ccd_sch_052_1415_w_0216161a_sas/ccd_sch_052_1415_w_0216161a.sas7bdat')

EmptyDataError: No columns to parse from file